In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sqlalchemy as sql
import pymysql
import sklearn.linear_model as skllm
import statsmodels.api as sm
import bin_time_data as bd
%matplotlib inline

In [2]:
sns.set_context('poster')

In [3]:
#connect to db
engine = sql.create_engine('mysql+pymysql://root:@localhost/zidisha')